This notebook provides a short and simple webcrawler for navigating over public domain websites and, if necessary, downloading pdf archives. It is useful for downloading articles about certain topics from public repositories. 
The program was created for 2 specific websites, so some changes are necessary when applied for other domains.

In [18]:
from datetime import date, datetime
import bs4 as bs
from selenium import webdriver
import time
import re
import os
import json
import schedule
## Need to install library and .exe file to be able to download pdf's    
## https://github.com/wkhtmltopdf/wkhtmltopdf/releases/download/0.12.5/wkhtmltox-0.12.5-1.mxe-cross-win64.7z
import pdfkit

working_dir = ''
os.chdir('')

options = webdriver.ChromeOptions()
options.add_argument('--disable-notifications')
options.add_argument('--no-sandbox')
options.add_argument('--verbose')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--incognito')
options.add_experimental_option('prefs',  {
    "download.default_directory": working_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,
    "safebrowsing.enabled": True,
    'safebrowsing.disable_download_protection': False,
})
driver=webdriver.Chrome(options=options)
driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': working_dir}}
command_result = driver.execute("send_command", params)

mapping = {
    'janeiro' : '01',
    'fevereiro' : '02',
    'março' : '03',
    'abril' : '04',
    'maio' : '05',
    'junho' : '06',
    'julho' : '07',
    'agosto' : '08',
    'setembro' : '09', 
    'outubro' : '10',
    'novembro' : '11',
    'dezembro' : '12'
}

os.getcwd()

'C:\\Users\\duhju\\Desktop\\Crawler'

In [2]:
def website_1(driver, url, date):    
    driver.get(url)
    ##Set the period to request the documents
    datepicker = driver.find_element_by_id("period")
    datepicker.click()
    datepicker = driver.find_element_by_id("init_date")
    datepicker.click()
    datepicker.clear()
    datepicker.send_keys(date)
    datepicker = driver.find_element_by_id("final_date")
    datepicker.click()
    datepicker.clear()
    datepicker.send_keys(date)
    search = driver.find_element_by_id("searcj")
    search.click()
    time.sleep(3)
    ##Find the links for each document and download their pdf's
    soup = bs.BeautifulSoup(driver.page_source, 'lxml')
    for tr in soup.find_all('table')[-1].find_all('tr')[1:]:
        driver.get('http://' + tr.find_all('a')[0].get('href'))
        driver.find_element_by_id("lbtnPdf").click()


In [3]:
def website_2(driver, url, year, month, today):
    driver = webdriver.Chrome(options=options)
    try:
        driver.get(url + year + '---' + month + '---')
    except:
        driver.get(url + '---' + year + '/' + month +'---')
    time.sleep(3)
    soup =  bs.BeautifulSoup(driver.page_source, 'lxml')
    links = []
    for tr in soup.find_all('table')[-1].find_all('tr')[1:3]:
        last_date = tr.find('td').get_text().split('-')[0].strip().split(' de ')
        if '/'.join([last_date[0], mapping[last_date[1]], last_date[2]]) == today:
            links.extend([i.get('href') for i in tr.find_all('a', attrs={'href': re.compile("^http://")})])
    for link in links:
        pdfkit.from_url(link, link.split('/')[-1].split('.')[0]+'.pdf')


In [ ]:
def main(today):
    try:
        website_1('http://',
                                  today.split('/')[2], today.split('/')[1], today)
        website_2(driver, 'http://', today)
        downloaded = True
    else:
        downloaded = False
    return downloaded

In [ ]:
while __name__ == "__main__":
    downloaded = False
    today = str(date.today() + timedelta(days=1))
    today = today.split('-')[2] + '/' + today.split('-')[1] + '/' + today.split('-')[0]
    time_now = str(datetime.now()).split()[1]
    if time_now >= "23:59:00.000000" and time_now < "23:59:59.999999":
        while not(downloaded):
            downloaded = main(today)
            time.sleep(60)
        time.sleep((23 - datetime.now().hour)*3600)
    